# 🧠 LangGraph + MCP + Ollama
## PLAN ↔ USER LOOP → ACT Autonomous Agent

Features:
- Infinite PLAN refinement
- Human approval gate
- Fully autonomous ACT
- MCP filesystem tools
- Notebook safe (no async servers)


In [ ]:
!pip install -U langgraph langchain langchain-ollama mcp

In [ ]:
from typing import TypedDict, List, Dict
from langgraph.graph import StateGraph, END
from langchain_ollama import ChatOllama
import os, json

In [ ]:
llm = ChatOllama(
    model="gpt-oss:20b",
    temperature=0
)

## Agent State

In [ ]:
class AgentState(TypedDict):
    goal: str
    plan: List[str]
    mode: str          # plan | act
    messages: List[Dict]
    done: bool

## MCP Filesystem Tools

In [ ]:
from mcp.server.fastmcp import FastMCP

BASE_DIR = "/tmp/mcp_workspace"
os.makedirs(BASE_DIR, exist_ok=True)

mcp = FastMCP("filesystem")

@mcp.tool()
def list_dir(path: str = "."):
    return os.listdir(os.path.join(BASE_DIR, path))

@mcp.tool()
def read_file(path: str) -> str:
    return open(os.path.join(BASE_DIR, path)).read()

@mcp.tool()
def write_file(path: str, content: str) -> str:
    with open(os.path.join(BASE_DIR, path), "w") as f:
        f.write(content)
    return "ok"

In [ ]:
def list_tools():
    return list(mcp._tools.keys())

def call_tool(name, args):
    return mcp._tools[name](**args)

## Planner Node (PLAN MODE)

In [ ]:
def planner_node(state: AgentState) -> AgentState:
    prompt = f"""
You are an AI planner.

User request:
{state['goal']}

Current plan:
{state['plan']}

Rules:
- If no plan exists, create one
- If plan exists, modify ONLY what user asked
- Do NOT execute anything
- Output numbered steps only
"""

plan_text = llm.invoke(prompt).content
state['plan'] = [l.strip() for l in plan_text.splitlines() if l.strip()]
state['done'] = False
return state

## Executor Node (ACT MODE)

In [ ]:
def executor_node(state: AgentState) -> AgentState:
    tools = list_tools()

    prompt = f"""
You are an autonomous agent.

Goal: {state['goal']}
Plan: {state['plan']}
History: {state['messages']}

Available tools:
{tools}

Decide ONE next action.

Return ONLY JSON:
{{"tool": "<tool|done>", "args": {{}}}}
"""

action = json.loads(llm.invoke(prompt).content)

if action['tool'] == 'done':
    state['done'] = True
    return state

state['messages'].append(action)
return state

In [ ]:
def tool_node(state: AgentState) -> AgentState:
    action = state['messages'][-1]
    result = call_tool(action['tool'], action.get('args', {}))
    state['messages'].append({'observation': result})
    return state

## PLAN Graph

In [ ]:
plan_graph = StateGraph(AgentState)
plan_graph.add_node("planner", planner_node)
plan_graph.set_entry_point("planner")
plan_graph.add_edge("planner", END)
plan_app = plan_graph.compile()

## ACT Graph

In [ ]:
act_graph = StateGraph(AgentState)
act_graph.add_node("executor", executor_node)
act_graph.add_node("tool", tool_node)
act_graph.set_entry_point("executor")
act_graph.add_conditional_edges(
    "executor",
    lambda s: END if s['done'] else 'tool'
)
act_graph.add_edge("tool", "executor")
act_app = act_graph.compile()

## 🧪 PLAN MODE Example

In [ ]:
state = {
    'goal': 'Read x1.txt and x2.txt and write a combined summary into summary.txt',
    'plan': [],
    'mode': 'plan',
    'messages': [],
    'done': False
}

state = plan_app.invoke(state)
state['plan']

## ✏️ User Modifies Plan

In [ ]:
state['goal'] = 'Change summary.txt to summ.txt'
state = plan_app.invoke(state)
state['plan']

## 🚀 ACT MODE

In [ ]:
state['mode'] = 'act'
state = act_app.invoke(state)
state['messages']